- https://www.pyquantnews.com/free-python-resources/event-driven-architecture-in-python-for-trading#:~:text=In%20the%20fast%2Dpaced%20realm,based%20on%20the%20latest%20information.
- https://github.com/topics/orderbook-tick-data
- https://medium.com/@gwrx2005/learning-without-retraining-implicit-in-context-learning-dynamics-for-adaptive-cryptocurrency-661ffeeaa489

Implementing a market-making simulation requires a more complex, event-driven framework than a simple backtest. It involves reconstructing an order book from historical tick data and processing events like new limit orders, cancellations, and trades as they occur. While a full-fledged simulation can be intricate, this code provides the core components for a simplified version. 

Prerequisites
For a real-world project, you'd need high-frequency, historical order book data, which can be acquired from data providers or cryptocurrency exchanges like Binance. This example uses simulated data for demonstration purposes. 

In [ ]:
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

class OrderBook:
    """
    A simplified Limit Order Book (LOB) class to manage bids and asks.
    """
    def __init__(self):
        self.bids = collections.OrderedDict()  # {price: quantity} - sorted descending
        self.asks = collections.OrderedDict()  # {price: quantity} - sorted ascending
    
    def add_order(self, order_id, side, price, quantity):
        if side == 'buy':
            if price in self.bids:
                self.bids[price] += quantity
            else:
                self.bids[price] = quantity
            # Sort bids descending
            self.bids = collections.OrderedDict(sorted(self.bids.items(), reverse=True))
        elif side == 'sell':
            if price in self.asks:
                self.asks[price] += quantity
            else:
                self.asks[price] = quantity
            # Sort asks ascending
            self.asks = collections.OrderedDict(sorted(self.asks.items()))

    def remove_order(self, order_id, side, price, quantity):
        if side == 'buy' and price in self.bids:
            self.bids[price] -= quantity
            if self.bids[price] <= 0:
                del self.bids[price]
        elif side == 'sell' and price in self.asks:
            self.asks[price] -= quantity
            if self.asks[price] <= 0:
                del self.asks[price]

    def process_trade(self, side, price, quantity):
        if side == 'buy':
            self.remove_order(None, 'sell', price, quantity)
        elif side == 'sell':
            self.remove_order(None, 'buy', price, quantity)

    def get_best_bid(self):
        return next(iter(self.bids.keys())) if self.bids else None
    
    def get_best_ask(self):
        return next(iter(self.asks.keys())) if self.asks else None

def simulate_market_making():
    """
    Simulates a simple market-making strategy based on a simplified order book.
    This example uses generated data to illustrate the process.
    In a real project, you would use historical order book tick data.
    """
    # --- Parameters for simulation ---
    num_steps = 1000
    asset_name = 'BTC-USD'
    inventory = 0  # Number of units of the asset held
    cash = 100000  # Starting cash in USD
    position = 0
    
    # --- Generate simulated order book data ---
    np.random.seed(42)
    mid_price_path = np.exp(np.random.randn(num_steps).cumsum() * 0.01) * 50000
    
    order_book = OrderBook()
    trades = []
    
    # --- Simulation loop ---
    for i in range(num_steps):
        mid_price = mid_price_path[i]
        
        # Add random bids and asks around the mid-price
        for _ in range(5):
            bid_price = mid_price * (1 - np.random.uniform(0.0001, 0.001))
            bid_qty = np.random.randint(1, 10)
            order_book.add_order(None, 'buy', bid_price, bid_qty)
            
            ask_price = mid_price * (1 + np.random.uniform(0.0001, 0.001))
            ask_qty = np.random.randint(1, 10)
            order_book.add_order(None, 'sell', ask_price, ask_qty)

        best_bid = order_book.get_best_bid()
        best_ask = order_book.get_best_ask()
        
        if best_bid is None or best_ask is None:
            continue
            
        # --- Market-Making Strategy ---
        # Place orders to capture the bid-ask spread
        my_bid_price = best_bid + 0.01
        my_ask_price = best_ask - 0.01
        
        # Check for fills (simplified)
        if best_ask <= my_bid_price:
            # My bid gets filled
            fill_price = best_ask
            fill_qty = np.random.randint(1, 3) # Random fill quantity
            cash -= fill_price * fill_qty
            inventory += fill_qty
            order_book.process_trade('buy', fill_price, fill_qty)
            trades.append({'time': i, 'price': fill_price, 'side': 'buy', 'qty': fill_qty})
            
        if best_bid >= my_ask_price:
            # My ask gets filled
            fill_price = best_bid
            fill_qty = np.random.randint(1, 3)
            cash += fill_price * fill_qty
            inventory -= fill_qty
            order_book.process_trade('sell', fill_price, fill_qty)
            trades.append({'time': i, 'price': fill_price, 'side': 'sell', 'qty': fill_qty})
        
        position = cash + inventory * mid_price
        
    # --- Backtest Summary ---
    print("--- Market Making Simulation Summary ---")
    print(f"Starting Cash: ${100000:.2f}")
    print(f"Ending Cash: ${cash:.2f}")
    print(f"Ending Inventory: {inventory:.4f} {asset_name}")
    print(f"Final Portfolio Value (Estimate): ${position:.2f}")

    # --- Plotting Results ---
    trade_prices = pd.DataFrame(trades)
    
    plt.figure(figsize=(15, 8))
    plt.plot(mid_price_path, label=f'Mid Price ({asset_name})')
    if not trade_prices.empty:
        buy_trades = trade_prices[trade_prices['side'] == 'buy']
        sell_trades = trade_prices[trade_prices['side'] == 'sell']
        plt.scatter(buy_trades['time'], buy_trades['price'], marker='^', color='green', label='Buy Fills', alpha=0.7)
        plt.scatter(sell_trades['time'], sell_trades['price'], marker='v', color='red', label='Sell Fills', alpha=0.7)
    
    plt.title(f'Market Making Strategy Simulation for {asset_name}')
    plt.xlabel('Time Step')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# Run the simulation
simulate_market_making()